In [1]:
%matplotlib inline
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA


In [2]:
#carrega o dataset
dados = pd.read_csv('/home/erasmor/Área de Trabalho/CSE-CIC-IDS2018/amostras_estratificadas/6amostragem_todas_instancias_somadas_baixa_alta_representatividade_estratificada.csv',sep=",",encoding = 'utf-8',  header=0,na_values='.',dtype={'Label':'category'})
#remove valores infinitos
dados.replace(-np.Inf, np.nan)
#substitui valores NaN
dados.fillna(dados.mean())


,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,53,17,391,1,1,46,62,46,46,46.000000,...,8,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Infilteration
1,53,17,37192,1,1,42,102,42,42,42.000000,...,8,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Infilteration
2,543,6,210,1,1,0,0,0,0,0.000000,...,24,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Infilteration
3,443,6,169,2,0,0,0,0,0,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Infilteration
4,3389,6,2810144,10,7,1122,1581,661,0,112.200000,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Infilteration
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303468,3389,6,3868068,14,8,1431,1727,725,0,102.214286,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Benign
303469,3389,6,3996691,15,8,1434,1727,725,0,95.600000,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Benign
303470,3389,6,3945545,14,8,1451,1727,741,0,103.642857,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Benign
303471,3389,6,3972090,14,8,1432,1727,725,0,102.285714,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Benign


In [3]:
#mostra como está a base de dados
dados.dtypes

Dst Port            int64
Protocol            int64
Flow Duration       int64
Tot Fwd Pkts        int64
Tot Bwd Pkts        int64
                   ...   
Idle Mean         float64
Idle Std          float64
Idle Max          float64
Idle Min          float64
Label            category
Length: 79, dtype: object

In [4]:
dados.memory_usage(deep=True)

Index                128
Dst Port         2427784
Protocol         2427784
Flow Duration    2427784
Tot Fwd Pkts     2427784
                  ...   
Idle Mean        2427784
Idle Std         2427784
Idle Max         2427784
Idle Min         2427784
Label             305124
Length: 80, dtype: int64

In [5]:
# verifica quantas instâncias (linhas) e quantos atributos (colunas) a base de dados contém
print("numero de linhas e colunas: ",dados.shape)

numero de linhas e colunas:  (303473, 79)


In [6]:
#visualizar distribições por classes contidas no csv - informar nome da classe alvo
print(dados.groupby('Label').size())

Label
Infilteration               17669
Benign                      64102
DDOS attack-HOIC            74854
DoS attacks-Hulk            50402
DoS attacks-SlowHTTPTest    15264
Bot                         31228
FTP-BruteForce              21099
SSH-Bruteforce              20469
DoS attacks-GoldenEye        4529
Brute Force -Web              611
Brute Force -XSS              230
DDOS attack-LOIC-UDP         1730
DoS attacks-Slowloris        1199
SQL Injection                  87
dtype: int64


In [7]:
#X_raw_normalize = MinMaxScaler(X_raw_normalize.reshape(0, 1)).reshape(len(X_raw_normalize))
#X_raw_normalizetd2 = (X_raw_normalize - X_raw_normalize.min(axis=0)) / (X_raw_normalize.max(axis=0) - X_raw_normalize.min(axis=0))
# Obtendo os nomes das colunas do DataFrame como uma lista.
cols = list(dados.columns)
# colunas que nao serao normalizadas
cols.remove('Label')
# Copiando os dados e aplicando a normalizacao por reescala nas colunas do DataFrame que contem
# valores continuos. Por padrao, o metodo minmax_scale reescala com min=0 e max=1.
dados = dados[~dados.isin([np.nan, np.inf, -np.inf]).any(1)]
dados[cols] = dados[cols].apply(minmax_scale)


In [8]:
#define as colunas de atributos e a coluna da classe (de 0 a 72 são atributos e após a 72 é a classe)
# "X_raw" é features/atributos e "y_raw" é target/classe ==> As duas formas abaixo dão certo.
#array = dataset.values
#X_raw = array[:,0:72]
#y_raw = array[:,72]
X_raw = dados.iloc[:, :-1].values # atributos
y_raw = dados.iloc[:, 78].values # classe de ataques
X_raw = np.nan_to_num(X_raw.astype(np.float32))


In [9]:
#transformar a variável Y com valores categóricos das classses de ataques em valores:
labelencoder_y = LabelEncoder()
y_raw = labelencoder_y.fit_transform(y_raw)


In [10]:
# Instanciando um PCA. O parametro n_components indica a quantidade de dimensoes que a base
# original sera reduzida.
pca = PCA(n_components=2, whiten=True,random_state=42)

In [11]:
# Aplicando o pca na base de dados. O atributo 'values' retorna um numpy.array
# de duas dimensões (matriz) contendo apenas os valores numericos do DataFrame.
X_raw = pca.fit_transform(X_raw)


In [12]:
def treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classifier):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    
    from xgboost import XGBClassifier
    from sklearn import svm
    from sklearn.svm import LinearSVC
    from sklearn.datasets import make_hastie_10_2
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
        
    # recupera amostras de teste de acordo com a dobra em uso
    X_test, y_test = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino de acordo com a dobra em uso
    X_train, y_train = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    performance_history = []

    #instancia classificadores de aprendizado
    if classifier == "knn":
        learner = KNeighborsClassifier(n_neighbors=5)
        arquivo_performance = open("baseline_performance_knn_dobra_"+indica_pool+".txt","a")
        titulo_grafico = 'Incremental classification accuracy KNN'
        
    if classifier == "rf":
        learner = RandomForestClassifier(random_state=42)
        arquivo_performance = open("baseline_performance_rf_dobra_"+indica_pool+".txt","a")
        titulo_grafico = 'Incremental classification accuracy RF'
        
    if classifier == "nb":
        learner = GaussianNB()
        arquivo_performance = open("baseline_performance_nb_dobra_"+indica_pool+".txt","a")
        titulo_grafico = 'Incremental classification accuracy NB'
        
    if classifier == "tree":
        learner = DecisionTreeClassifier()
        arquivo_performance = open("baseline_performance_tree_dobra_"+indica_pool+".txt","a")
        titulo_grafico = 'Incremental classification accuracy TREE'
        
    if classifier == "mlp":
        learner = MLPClassifier(max_iter=2000)
        arquivo_performance = open("baseline_performance_mlp_dobra_"+indica_pool+".txt","a")
        titulo_grafico = 'Incremental classification accuracy MLP'
    
    if classifier == "xgb":
        learner = GradientBoostingClassifier(random_state=42)
        arquivo_performance = open("baseline_performance_xgb_dobra_"+indica_pool+".txt","a")
        titulo_grafico = 'Incremental classification accuracy xgb'
    if classifier == "svm":
        learner = svm.SVC(kernel='linear',probability=True)
        arquivo_performance = open("baseline_performance_svm_dobra_"+indica_pool+".txt","a")
        titulo_grafico = 'Incremental classification accuracy svm'
      
    # treina o modelo
    learner.fit(X_train, y_train)
    predictions=learner.predict(X_test)
        
    #print('Accuracy: %f' % accuracy_score(y_test,predictions))
    arquivo_performance.write('Accuracy %f \n' % accuracy_score(y_test,predictions))
    #print ('Precision:  %f' % precision_score(y_test, predictions,average='macro',zero_division=1))
    arquivo_performance.write('Precision: %f \n' % precision_score(y_test, predictions,average='macro',zero_division=1))
    #print ('F1 score: %f' % f1_score(y_test, predictions,average='macro',zero_division=1))
    arquivo_performance.write('Recall: %f \n' % recall_score(y_test, predictions, average='macro',zero_division=1))
    #arquivo_performance.write('F1 score: %f \n' % f1_score(y_test, predictions,average='macro',zero_division=1))
    #print ('Recall: %f' % recall_score(y_test, predictions, average='macro',zero_division=1))
    f1score= 2*((precision_score(y_test, predictions,average='macro',zero_division=1)*recall_score(y_test, predictions, average='macro',zero_division=1))/(precision_score(y_test, predictions,average='macro',zero_division=1)+recall_score(y_test, predictions, average='macro',zero_division=1)))
    arquivo_performance.write('F1 score: %f \n' % f1score)
    #print("========================================")
    arquivo_performance.write("\n Avaliação por classe \n")
    arquivo_performance.write(classification_report(y_test, predictions,zero_division=1))
    arquivo_performance.close()

    
      

import time
import sys
import threading
from datetime import datetime
from datetime import date

from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, train_test_split
from modAL.uncertainty import classifier_uncertainty
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from timeit import Timer
import time
import functools

#inicia relogio
t1 = time.time()
# Define o tamanho das divisões feitas no dataset (cross-validation)
n_splits = 10
#define array de performance
performance_history = []
#define array de indices das partições
idx_data =[]
# cross validation bags - n_splits
data_cv = StratifiedShuffleSplit(n_splits= n_splits, train_size=350,test_size=350, random_state=42) 
data_cv.get_n_splits(X_raw, y_raw)
# chame a instância e gere os dados sobre a base original
type(data_cv.split(X_raw, y_raw))
# dividir os dados - A função split.split () retorna índices para amostras de treino e amostras de teste. 
# Ele examinará o número de validação cruzada especificado e retornará cada vez que treinar 
# e testar os índices de amostra usando os conjuntos de dados de treinamento e teste que podem 
# ser criados filtrando o conjunto de dados inteiro. Por exemplo idx_data[0][1], o primeiro indice faz referencia
# a dobra e o segundo indice faz referencia a posição da dobra (0 = treino e 1 = teste)
for train_index, test_index in data_cv.split(X_raw,y_raw):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("n_split",n_splits,"TRAIN:", train_index, "TEST:", test_index)
    idx_data.append([train_index, test_index])
#verifica tamanho das dobras (numero de instâncias de cada dobra)
#print("tamanho de cada dobra: ",idx_data[3][0].shape)

TRAIN =0
TEST =1
#chama procedimento de aprendizado para todas as dobras

classificador_knn="knn"
classificador_rf="rf"
classificador_nb="nb"
classificador_tree="tree"
classificador_mlp="mlp"
classificador_xgb="xgb"
classificador_svm="svm"

#classificadores=['knn','rf','nb','tree']
#for classificador in classificadores:
#    classifier = classificador
#    for idx_dobra in range(n_splits):
        #treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classifier)
#for idx_dobra in range(n_splits):
for idx_dobra in range(1):    
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_knn)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_rf)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_nb)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_mlp)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_xgb)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_tree)).start()
    threading.Thread(target=treino(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_svm)).start()
    
    
t2 = time.time()
time_elapsed = (t2-t1)/3600
print("tempo: ",time_elapsed)

/home/erasmor/ppca/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


tempo:  0.002352391547626919
